In [0]:
%sql
USE itversity_retail_db;

In [0]:
%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users(
  user_id INT,
  user_fname STRING,
  user_lname STRING,
  user_phones ARRAY<STRING>
);

INSERT INTO users VALUES 
(1, 'Scott', 'Tiger', ARRAY('+1 (234) 567 8901', '+1 (123) 456 7890')),
(2, 'Donald', 'Duck', NULL),
(3, 'Mickey', 'Mouse', ARRAY('+1 (456) 789 0123'));

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"List(+1 (234) 567 8901, +1 (123) 456 7890)"
2,Donald,Duck,null
3,Mickey,Mouse,List(+1 (456) 789 0123)


#### Dealing with ARRAY Type Columns
- `size`, `explode`, `explode_outer`

In [0]:
%sql
SELECT user_id, size(user_phones) as phone_count
FROM users;

user_id,phone_count
1,2
2,-1
3,1


In [0]:
%sql
-- Ex1. SIZE can be used to get count from the ARRAY
SELECT user_id
, CASE WHEN user_phones IS NOT NULL THEN size(user_phones) ELSE 0 END as phone_count
FROM users;

user_id,phone_count
1,2
2,0
3,1


In [0]:
%sql

-- Ex2. EXPLODE flattens data. Excludes nulls
SELECT user_id
, explode(user_phones) as user_phone
FROM users;

-- flatten means elements of an ARRAY into individual records
-- Note: can be used only with SELECT clause, use Nested query/CTE is you have to filter, or to GROUP BY

user_id,user_phone
1,+1 (234) 567 8901
1,+1 (123) 456 7890
3,+1 (456) 789 0123


In [0]:
%sql
-- Ex3. EXPLODE_OUTER flattens data. Includes nulls
SELECT user_id
, explode_outer(user_phones) as user_phone
FROM users;

user_id,user_phone
1,+1 (234) 567 8901
1,+1 (123) 456 7890
2,null
3,+1 (456) 789 0123


#### Dealing with STRUCT Type Column

In [0]:
%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users(
  user_id INT,
  user_fname STRING,
  user_lname STRING,
  user_phones STRUCT<home:STRING, mobile:STRING>
);

INSERT INTO users VALUES 
(1, 'Scott', 'Tiger', STRUCT('+1 (234) 567 8901', '+1 (123) 456 7890')),
(2, 'Donald', 'Duck', STRUCT(NULL, NULL)),
(3, 'Mickey', 'Mouse', STRUCT('+1 (456) 789 0123', NULL));

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"List(+1 (234) 567 8901, +1 (123) 456 7890)"
2,Donald,Duck,"List(null, null)"
3,Mickey,Mouse,"List(+1 (456) 789 0123, null)"


In [0]:
%sql
--Ex1:  .* will flatten the STRUCT column data in multiple columns
SELECT user_id, user_phones.* 
FROM users;

user_id,home,mobile
1,+1 (234) 567 8901,+1 (123) 456 7890
2,null,null
3,+1 (456) 789 0123,null


In [0]:
%sql
-- Ex2: Access individual STRUCT column data using dot .
SELECT user_id
, user_phones.home
, user_phones.mobile
FROM users;

user_id,home,mobile
1,+1 (234) 567 8901,+1 (123) 456 7890
2,null,null
3,+1 (456) 789 0123,null


In [0]:
%sql
--Ex3: Sometimes you might wanna build STRUCT Type field from regular columns 
SELECT * FROM order_items LIMIT 5;

order_item,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99


In [0]:
%sql
SELECT order_item,
       order_item_order_id,
       order_item_product_id,
       order_item_subtotal,
       STRUCT(order_item_quantity, order_item_product_price) as order_item_trans_details
FROM order_items limit 5;

order_item,order_item_order_id,order_item_product_id,order_item_subtotal,order_item_trans_details
1,1,957,299.98,"List(1, 299.98)"
2,2,1073,199.99,"List(1, 199.99)"
3,2,502,250.0,"List(5, 50.0)"
4,2,403,129.99,"List(1, 129.99)"
5,4,897,49.98,"List(2, 24.99)"


In [0]:
%sql
--Ex4: In some cases we might have to create table with ARRAY(STRUCT) Type field 
DROP TABLE IF EXISTS users;
CREATE TABLE users(
  user_id INT,
  user_fname STRING,
  user_lname STRING,
  user_phones ARRAY<STRUCT<phone_type:STRING, phone_number:STRING>>
);

INSERT INTO users VALUES 
(1, 'Scott', 'Tiger', ARRAY(STRUCT('home', '+1 (234) 567 8901'), STRUCT('mobile',  '+1 (123) 456 7890'))),
(2, 'Donald', 'Duck', NULL),
(3, 'Mickey', 'Mouse', ARRAY(STRUCT('home', '+1 (123) 456 9012')));

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"List(List(home, +1 (234) 567 8901), List(mobile, +1 (123) 456 7890))"
2,Donald,Duck,null
3,Mickey,Mouse,"List(List(home, +1 (123) 456 9012))"


#### Dealing with ARRAY(STRUCT) Type field

In [0]:
%sql
WITH cte AS 
(SELECT user_id
     , explode_outer(user_phones) as user_phone -- Note: you cannot use .* with explode function
FROM users)
SELECT  user_id
      , user_phone.*  -- or user_phone.phone_type, user_phone.phone_number
FROM cte

user_id,phone_type,phone_number
1,home,+1 (234) 567 8901
1,mobile,+1 (123) 456 7890
2,null,null
3,home,+1 (123) 456 9012


#### Important Functions to Process JSON Data in Spark SQL
- `split`, `concat_ws`, `collect_*`, `array_*`

In [0]:
%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users(
  user_id INT,
  user_fname STRING,
  user_lname STRING,
  user_phone_type STRING,
  user_phone_number STRING
);

INSERT INTO users VALUES 
(1, 'Scott', 'Tiger', 'home', '+1 (234) 567 8901'),
(1, 'Scott', 'Tiger', 'mobile', '+1 (123) 456 7890'),
(2, 'Donald', 'Duck', NULL, NULL),
(3, 'Mickey', 'Mouse', 'home', '+1 (123) 456 9012');

SELECT * FROM users;


user_id,user_fname,user_lname,user_phone_type,user_phone_number
1,Scott,Tiger,home,+1 (234) 567 8901
1,Scott,Tiger,mobile,+1 (123) 456 7890
2,Donald,Duck,null,null
3,Mickey,Mouse,home,+1 (123) 456 9012


In [0]:
%sql
-- Requirement:
-- 1 Scott Tiger [+1 (234) 567 8901, +1 (123) 456 7890]
-- 2 Donald Duck NULL
-- 3 Mickey Mouse [+1 (123) 456 9012]

In [0]:
%sql
SELECT user_id
, user_fname
, user_lname
, count(*) as phone_count
FROM users
GROUP BY 1, 2, 3;

user_id,user_fname,user_lname,phone_count
2,Donald,Duck,1
1,Scott,Tiger,2
3,Mickey,Mouse,1


In [0]:
%sql
-- Ex1: COLLECT_LIST retains duplicates & COLLECT_SET ignores duplicate 
SELECT user_id
, user_fname
, user_lname
, collect_list(user_phone_number) as user_phones
FROM users
GROUP BY 1, 2, 3;

user_id,user_fname,user_lname,user_phones
2,Donald,Duck,List()
1,Scott,Tiger,"List(+1 (234) 567 8901, +1 (123) 456 7890)"
3,Mickey,Mouse,List(+1 (123) 456 9012)


In [0]:
%sql
-- Ex2: If you want to represent in delimitted string
SELECT user_id
, user_fname
, user_lname
, concat_ws(',', collect_list(user_phone_number)) as user_phones
FROM users
GROUP BY 1, 2, 3;


user_id,user_fname,user_lname,user_phones
2,Donald,Duck,
1,Scott,Tiger,"+1 (234) 567 8901,+1 (123) 456 7890"
3,Mickey,Mouse,+1 (123) 456 9012


In [0]:
%sql
-- Ex3. If you want to convert to Struct Type
SELECT user_id
, user_fname
, user_lname
, STRUCT(user_phone_type, user_phone_number) as user_phones
FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"List(home, +1 (234) 567 8901)"
1,Scott,Tiger,"List(mobile, +1 (123) 456 7890)"
2,Donald,Duck,"List(null, null)"
3,Mickey,Mouse,"List(home, +1 (123) 456 9012)"


In [0]:
%sql
SELECT user_id
, user_fname
, user_lname
, CASE WHEN user_phone_number IS NOT NULL THEN STRUCT(user_phone_type, user_phone_number) ELSE NULL END as user_phones
FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"List(home, +1 (234) 567 8901)"
1,Scott,Tiger,"List(mobile, +1 (123) 456 7890)"
2,Donald,Duck,null
3,Mickey,Mouse,"List(home, +1 (123) 456 9012)"


In [0]:
%sql
-- Ex4. If you want to convert into ARRAY(STRUCT) Type
SELECT user_id
, user_fname
, user_lname
,  collect_list(CASE WHEN user_phone_number IS NOT NULL THEN STRUCT(user_phone_type, user_phone_number) ELSE NULL END) as user_phones
FROM users
GROUP BY 1, 2, 3;

user_id,user_fname,user_lname,user_phones
2,Donald,Duck,List()
1,Scott,Tiger,"List(List(home, +1 (234) 567 8901), List(mobile, +1 (123) 456 7890))"
3,Mickey,Mouse,"List(List(home, +1 (123) 456 9012))"


In [0]:
%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users(
  user_id INT,
  user_fname STRING,
  user_lname STRING,
  user_phones STRING
);

INSERT INTO users VALUES 
(1, 'Scott', 'Tiger', '+1 (234) 567 8901, +1 (123) 456 7890'),
(2, 'Donald', 'Duck', NULL),
(3, 'Mickey', 'Mouse', '+1 (456) 789 0123');

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,"+1 (234) 567 8901, +1 (123) 456 7890"
2,Donald,Duck,null
3,Mickey,Mouse,+1 (456) 789 0123


In [0]:
%sql
SELECT
user_id
, user_fname
, user_lname
, explode_outer(split(user_phones, ',')) AS user_phones -- split will convert in ARRAY, EXPLODE will flatten
 FROM users;

user_id,user_fname,user_lname,user_phones
1,Scott,Tiger,+1 (234) 567 8901
1,Scott,Tiger,+1 (123) 456 7890
2,Donald,Duck,null
3,Mickey,Mouse,+1 (456) 789 0123
